In [11]:
from elasticsearch import Elasticsearch

In [12]:
'''
Parent-join restrictions
    Only one join field mapping is allowed per index.
    Parent and child documents must be indexed on the same shard. This means that the same routing value needs to be provided when getting, deleting, or updating a child document.
    An element can have multiple children but only one parent.
    It is possible to add a new relation to an existing join field.
    It is also possible to add a child to an existing element but only if the element is already a parent.
'''
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
body = {
    "mappings": {
        "properties": {
            "name": {
                "type": "keyword"
            },
            "my-join-field": {  # 字段名称
                "type": "join",
                "relations": {
                    "class": "student" # Defines a single relation where class is parent of class.
                    # it is also possible to define multiple children for a single parent:
                    # "class": ["student", "teacher"]
                }
            }
        }
    }
}
es.indices.create(index='es0', body=body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [13]:
es.index(index='es0', id=0, body={
    "name": "一班",
    "my-join-field": {
        "name": "class"  # 父文档
    }
})
es.index(index='es0', id=1, body={
    "name": "二班",
    "my-join-field": {
        "name": "class"
    }
})
es.index(index='es0', id=2, body={
    "name": "xiaoming",
    "my-join-field": {
        "name": "student",  # 子文档
        "parent": 0  # The parent id of this child document
    }
},
         # The routing value is mandatory because parent and child documents must be indexed on the same shard
         routing=0)

es.index(index='es0', id=3, body={
    "name": "xiaoli",
    "my-join-field": {
        "name": "student",
        "parent": 0
    }
},
         routing=0)

es.index(index='es0', id=4, body={
    "name": "xiahei",
    "my-join-field": {
        "name": "student",
        "parent": 1
    }
},
         routing=1)

{'_index': 'es0',
 '_type': '_doc',
 '_id': '4',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 4,
 '_primary_term': 1}

In [14]:
body = {
    "query": {
        "has_child": {
            "type": "student",
            "query": {
                "match": {
                    "name": "xiaoli"
                }
            }
        }
    }
}
'''
GET test/_search
`body`
'''
# Returns parent documents whose joined child documents match a provided query.
es.search(index='es0', body=body)

{'took': 21,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 1.0,
    '_source': {'name': '一班', 'my-join-field': {'name': 'class'}}}]}}

In [15]:
# Returns child documents whose joined parent document matches a provided query.
es.search(index='es0', body={
    "query": {
        "has_parent": {
            "parent_type": "class",
            "query": {
                "match": {
                    "name": "一班"
                }
            }
        }
    }
})

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.0,
    '_routing': '0',
    '_source': {'name': 'xiaoming',
     'my-join-field': {'name': 'student', 'parent': 0}}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3',
    '_score': 1.0,
    '_routing': '0',
    '_source': {'name': 'xiaoli',
     'my-join-field': {'name': 'student', 'parent': 0}}}]}}

In [10]:
es.indices.delete('es0')

{'acknowledged': True}